## Importing all required libraries

In [31]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

## Repeating the work done in 1st part of the assignment to build the dataframe by scraping Wikipedia page

In [106]:

from bs4 import BeautifulSoup
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
indexNames = df[ df['Borough'] == 'Not assigned'].index
# Delete these row indexes from dataFrame
df.drop(indexNames, inplace=True)
df.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [107]:
df.shape

(103, 3)

In [109]:
# Adding extra columns for longitude and latitude
df['Latitude'] = 'default value'
df['Longitude'] = 'default value'
#df.set_index('Neighborhood')
df.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,default value,default value
3,M4A,North York,Victoria Village,default value,default value
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",default value,default value
5,M6A,North York,"Lawrence Manor, Lawrence Heights",default value,default value
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",default value,default value
8,M9A,Etobicoke,Islington Avenue,default value,default value
9,M1B,Scarborough,"Malvern, Rouge",default value,default value
11,M3B,North York,Don Mills,default value,default value
12,M4B,East York,"Parkview Hill, Woodbine Gardens",default value,default value
13,M5B,Downtown Toronto,"Garden District, Ryerson",default value,default value


## Using Geopy API to get the longitude and latitude values for the neighborhoods in Toronto

In [110]:
for index, row in df.iterrows():
    geolocator = Nominatim(user_agent="toronto_explorer")
    address = row['Neighborhood']
    a = address + ", Toronto"
    print(index)
    print(a)
    location = geolocator.geocode(a)
    if location is None:
        print("Lets move on")
    else:
        print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
        df.loc[index,'Latitude'] = location.latitude
        df.loc[index,'Longitude'] = location.longitude

2
Parkwoods, Toronto
Latitude = 43.7587999, Longitude = -79.3201966
3
Victoria Village, Toronto
Latitude = 43.732658, Longitude = -79.3111892
4
Regent Park, Harbourfront, Toronto
Latitude = 43.64076885, Longitude = -79.37989177980148
5
Lawrence Manor, Lawrence Heights, Toronto
Latitude = 43.7152827, Longitude = -79.4439143
6
Queen's Park, Ontario Provincial Government, Toronto
Lets move on
8
Islington Avenue, Toronto
Latitude = 43.6794838, Longitude = -79.5389092
9
Malvern, Rouge, Toronto
Latitude = 43.8091955, Longitude = -79.2217008
11
Don Mills, Toronto
Latitude = 43.775347, Longitude = -79.3459439
12
Parkview Hill, Woodbine Gardens, Toronto
Lets move on
13
Garden District, Ryerson, Toronto
Lets move on
14
Glencairn, Toronto
Latitude = 43.7087117, Longitude = -79.4406853
17
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale, Toronto
Lets move on
18
Rouge Hill, Port Union, Highland Creek, Toronto
Lets move on
20
Don Mills, Toronto
Latitude = 43.775347, Longitude =

In [111]:
df_ready = df[df['Longitude']!='default value']
df_missing = df[df['Longitude']=='default value']
print(df_ready.shape)
print(df_missing.shape)

(59, 5)
(44, 5)


## I have used Geopy to get the coordinates to the neighborhoods in Toronto, but for caes where there are more than 2 neighborhoods in the same postcode, Geopy has not repturned values. So we have coordinates for 59 and missing for 44 records.

## I will now use the file given in the assignment to prepare the dataframe

In [112]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
df_downloaded = pd.read_csv('Geospatial_Coordinates.csv')
df_downloaded.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Since the loaded file is sorted by the Postal Code, in below cell I am sorting my original Dataframe df in ascending order of the postal code

In [113]:
df.sort_values(by=['Postal Code'], inplace=True)
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
9,M1B,Scarborough,"Malvern, Rouge",43.8092,-79.2217
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",default value,default value
27,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7535,-79.18
36,M1G,Scarborough,Woburn,43.7598,-79.2253
45,M1H,Scarborough,Cedarbrae,43.7565,-79.2267


## In below cell, using a for loop, I am interating through the dataframe rows, and wherever the Latitude and Longitude values are missing (did not get from Geopy), I am copying the values from the loaded dataframe

In [121]:
for index, row in df.iterrows():
    counter = 0
    postcode = row['Postal Code']
    print(index)
    print(postcode)
    if df.loc[index,'Latitude']=='default value':
        print("Have to copy data")
        counter = counter + 1
        df.loc[index,'Latitude'] = df_downloaded.loc[counter,'Latitude']
        df.loc[index,'Longitude'] = df_downloaded.loc[counter,'Longitude']
    else:
        print("Data present")        

9
M1B
Data present
18
M1C
Have to copy data
27
M1E
Data present
36
M1G
Data present
45
M1H
Data present
54
M1J
Data present
63
M1K
Have to copy data
72
M1L
Have to copy data
81
M1M
Have to copy data
90
M1N
Have to copy data
99
M1P
Have to copy data
108
M1R
Data present
117
M1S
Data present
126
M1T
Have to copy data
135
M1V
Have to copy data
144
M1W
Data present
153
M1X
Data present
46
M2H
Data present
55
M2J
Have to copy data
64
M2K
Data present
73
M2L
Data present
82
M2M
Have to copy data
91
M2N
Data present
100
M2P
Data present
109
M2R
Data present
2
M3A
Data present
11
M3B
Data present
20
M3C
Data present
47
M3H
Have to copy data
56
M3J
Have to copy data
65
M3K
Data present
74
M3L
Data present
83
M3M
Data present
92
M3N
Data present
3
M4A
Data present
12
M4B
Have to copy data
21
M4C
Data present
30
M4E
Data present
39
M4G
Data present
48
M4H
Data present
57
M4J
Data present
66
M4K
Data present
75
M4L
Data present
84
M4M
Data present
93
M4N
Data present
102
M4P
Data present
111
M4R
D

In [122]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
9,M1B,Scarborough,"Malvern, Rouge",43.8092,-79.2217
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7845,-79.1605
27,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7535,-79.18
36,M1G,Scarborough,Woburn,43.7598,-79.2253
45,M1H,Scarborough,Cedarbrae,43.7565,-79.2267


In [137]:
df.sort_index(inplace=True)
df.reset_index(inplace = True) 
df.head(10)

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.7588,-79.3202
1,3,M4A,North York,Victoria Village,43.7327,-79.3112
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6408,-79.3799
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7153,-79.4439
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.7845,-79.1605
5,8,M9A,Etobicoke,Islington Avenue,43.6795,-79.5389
6,9,M1B,Scarborough,"Malvern, Rouge",43.8092,-79.2217
7,11,M3B,North York,Don Mills,43.7753,-79.3459
8,12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7845,-79.1605
9,13,M5B,Downtown Toronto,"Garden District, Ryerson",43.7845,-79.1605


## The Dataframe is now ready!

In [138]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
counter = 0
# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    counter = counter + 1
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
print (counter)
map_Toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.
103


## Defining Foursqaure credentials

In [139]:
CLIENT_ID = '0IGO0GF0MHBANY0JGUWRYXQHVVZZHZ23OZKSL5NA1HOQBNW1' # your Foursquare ID
CLIENT_SECRET = 'FIDY1VPGAN55RLWQXJQHN02UDGUDTU3VGWVXFHA2Q5N5QJA5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0IGO0GF0MHBANY0JGUWRYXQHVVZZHZ23OZKSL5NA1HOQBNW1
CLIENT_SECRET:FIDY1VPGAN55RLWQXJQHN02UDGUDTU3VGWVXFHA2Q5N5QJA5


In [146]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    Counter = 0
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(Counter, name)
        Counter = Counter + 1
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [147]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

0 Parkwoods
1 Victoria Village
2 Regent Park, Harbourfront
3 Lawrence Manor, Lawrence Heights
4 Queen's Park, Ontario Provincial Government
5 Islington Avenue
6 Malvern, Rouge
7 Don Mills
8 Parkview Hill, Woodbine Gardens
9 Garden District, Ryerson
10 Glencairn
11 West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
12 Rouge Hill, Port Union, Highland Creek
13 Don Mills
14 Woodbine Heights
15 St. James Town
16 Humewood-Cedarvale
17 Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
18 Guildwood, Morningside, West Hill
19 The Beaches
20 Berczy Park
21 Caledonia-Fairbanks
22 Woburn
23 Leaside
24 Central Bay Street
25 Christie
26 Cedarbrae
27 Hillcrest Village
28 Bathurst Manor, Wilson Heights, Downsview North
29 Thorncliffe Park
30 Richmond, Adelaide, King
31 Dufferin, Dovercourt Village
32 Scarborough Village
33 Fairview, Henry Farm, Oriole
34 Northwood Park, York University
35 East Toronto
36 Harbourfront East, Union Station, Toronto Islands
37 Little Portu

In [148]:
print(toronto_venues.shape)
toronto_venues.head()

(1935, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7588,-79.320197,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.7588,-79.320197,LCBO,43.757774,-79.314257,Liquor Store
2,Parkwoods,43.7588,-79.320197,Petro-Canada,43.757950,-79.315187,Gas Station
3,Parkwoods,43.7588,-79.320197,Shoppers Drug Mart,43.760857,-79.324961,Pharmacy
4,Parkwoods,43.7588,-79.320197,Pizza Pizza,43.760231,-79.325666,Pizza Place


In [149]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,14,14,14,14,14,14
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",2,2,2,2,2,2
Bayview Village,11,11,11,11,11,11
"Bedford Park, Lawrence Manor East",2,2,2,2,2,2
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",2,2,2,2,2,2
"Brockton, Parkdale Village, Exhibition Place",2,2,2,2,2,2
Business reply mail Processing Centre,2,2,2,2,2,2


In [150]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


In [151]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [154]:
toronto_onehot.shape

(1935, 235)

In [155]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.071429,0.0000,0.000000,0.000000,0.000000
1,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
3,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
4,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
5,Berczy Park,0.010000,0.000000,0.000000,0.010000,0.01,0.0,0.020000,0.000000,0.000000,...,0.00,0.00,0.010000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
6,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
8,Business reply mail Processing Centre,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000


In [156]:
toronto_grouped.shape

(98, 235)

In [157]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                    venue  freq
0      Chinese Restaurant  0.14
1        Asian Restaurant  0.14
2  Peking Duck Restaurant  0.07
3     Rental Car Location  0.07
4           Shopping Mall  0.07


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1             Pub  0.14
2  Sandwich Place  0.14
3     Coffee Shop  0.14
4        Pharmacy  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
                           venue  freq
0                            Bar   0.5
1                   Home Service   0.5
2  Paper / Office Supplies Store   0.0
3                         Museum   0.0
4                   Music School   0.0


----Bayview Village----
                  venue  freq
0                  Bank  0.18
1           Gas Station  0.09
2  Outdoor Supply Store  0.09
3           Fish Market  0.09
4        Sandwich Place  0.09


----Bedford Park, Lawrence Manor East----
                           venue  freq
0                            Bar

In [158]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [160]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Asian Restaurant,Coffee Shop,Train Station,Shopping Mall,Cantonese Restaurant,Rental Car Location,Korean Restaurant,Food Court,Vietnamese Restaurant
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Pub,Pharmacy,Gym,Women's Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bar,Home Service,Women's Store,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Filipino Restaurant
3,Bayview Village,Bank,Pizza Place,Outdoor Supply Store,Breakfast Spot,Fish Market,Fast Food Restaurant,Gas Station,Sandwich Place,Sporting Goods Shop,Persian Restaurant
4,"Bedford Park, Lawrence Manor East",Bar,Home Service,Women's Store,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Filipino Restaurant


In [161]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 1, 0, 1, 1, 1, 1], dtype=int32)

In [162]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6408,-79.3799,0,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Bank,Brewery,Plaza,Chinese Restaurant,Pizza Place
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.7845,-79.1605,1,Bar,Home Service,Women's Store,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Filipino Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.7845,-79.1605,1,Bar,Home Service,Women's Store,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Filipino Restaurant
3,M5C,Downtown Toronto,St. James Town,43.6694,-79.3727,0,Coffee Shop,Pizza Place,Grocery Store,Café,Hotel,Playground,Diner,Restaurant,Filipino Restaurant,Caribbean Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.648,-79.3754,0,Coffee Shop,Café,Restaurant,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Beer Bar,Cocktail Bar,Gym,Breakfast Spot


In [163]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [164]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Bank,Brewery,Plaza,Chinese Restaurant,Pizza Place
3,Downtown Toronto,0,Coffee Shop,Pizza Place,Grocery Store,Café,Hotel,Playground,Diner,Restaurant,Filipino Restaurant,Caribbean Restaurant
4,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Beer Bar,Cocktail Bar,Gym,Breakfast Spot
5,Downtown Toronto,0,Coffee Shop,Clothing Store,Hotel,Diner,Cosmetics Shop,Restaurant,Sandwich Place,Bubble Tea Shop,Middle Eastern Restaurant,Plaza
6,Downtown Toronto,0,Korean Restaurant,Coffee Shop,Indian Restaurant,Mexican Restaurant,Karaoke Bar,Dessert Shop,Cocktail Bar,Café,Grocery Store,Sandwich Place
10,Downtown Toronto,0,Coffee Shop,Restaurant,Hotel,Seafood Restaurant,Café,American Restaurant,Gym,Japanese Restaurant,Italian Restaurant,Gastropub
11,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Italian Restaurant,Japanese Restaurant,Gym,Bakery,Bookstore,Yoga Studio,Comfort Food Restaurant
14,Downtown Toronto,0,Playground,Bike Trail,Park,Building,Women's Store,Event Space,Food Service,Food Court,Food & Drink Shop,Food
18,Downtown Toronto,0,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Café,Restaurant,Yoga Studio,Gastropub,Gay Bar,Men's Store,Mediterranean Restaurant


In [165]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,1,Bar,Home Service,Women's Store,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Filipino Restaurant
2,Downtown Toronto,1,Bar,Home Service,Women's Store,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Filipino Restaurant
7,Downtown Toronto,1,Bar,Home Service,Women's Store,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Filipino Restaurant
8,Downtown Toronto,1,Bar,Home Service,Women's Store,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Filipino Restaurant
9,Downtown Toronto,1,Bar,Home Service,Women's Store,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Filipino Restaurant
12,Downtown Toronto,1,Bar,Home Service,Women's Store,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Filipino Restaurant
13,Downtown Toronto,1,Bar,Home Service,Women's Store,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Filipino Restaurant
15,Downtown Toronto,1,Bar,Home Service,Women's Store,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Filipino Restaurant
16,Downtown Toronto,1,Bar,Home Service,Women's Store,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Filipino Restaurant
17,Downtown Toronto,1,Bar,Home Service,Women's Store,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Filipino Restaurant


In [168]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
